In [ ]:
import os
import pandas as pd
import numpy as np
import json
import pickle
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from models.import_plotting import *

from models.types import *
from train.utils import *
from train.predictors import *

DS_FOLDER = f"{HOME_PATH}/data/rand"


## Create Dataset

In [ ]:
runs = get_runs(DS_FOLDER)

In [ ]:
dps = get_datapoints(runs, use_gpu=True, augment_zero_pred=False)

## Visualize

In [ ]:
data = []

for dp in dps:
    d = {
        "id": dp.x.id,
        "application": dp.x.application,
        "service_time_m": dp.x.service_time_m,
        "service_time_p75": dp.x.service_time_p75,
        "service_time_p95": dp.x.service_time_p95,
    }

    for metric in dp.x.low_metrics:
        d[metric.name] = metric.mean
    
    data.append(d)

In [ ]:
data_df = pd.DataFrame(data)
data_df = data_df.groupby(by="application")

In [ ]:
sns.histplot(data=data_df.get_group("tstr-lstm"), x="service_time_m")


## Stats

In [ ]:
stats_min = {}
stats_max = {}

for app, df in data_df:
    sm_min = df["service_time_m"].min()
    sm_max = df["service_time_m"].max()

    sp75_min = df["service_time_p75"].min()
    sp75_max = df["service_time_p75"].max()

    sp95_min = df["service_time_p95"].min()
    sp95_max = df["service_time_p95"].max()

    stats_min[app] = [sm_min, sp75_min, sp95_min]
    stats_max[app] = [sm_max, sp75_max, sp95_max]


stats = {
    "min": stats_min,
    "max": stats_max,
}

json.dump(stats, open(f"{HOME_PATH}/models/sts_server1.json", "w"), indent=4)